In [ ]:
!pip install -q datasets transformers evaluate torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q --upgrade transformers accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 37.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

In [ ]:
# Importação das bibliotecas necessárias
import torch
import pandas as pd
import numpy as np
import evaluate
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split


In [1]:
import pandas as pd

In [ ]:
# Montar o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Definir um diretório dentro do Google Drive para salvar o modelo
OUTPUT_DIR = "/content/drive/MyDrive/BERTimbau_Sentimento_Sexto"

In [ ]:
# Definição de parâmetros
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"  # Modelo BERTimbau-base
MAX_SEQ_LENGTH = 64  # Baseado na análise dos tokens
BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 2e-5

In [2]:
# Carregar o dataset
df = pd.read_csv("df_bancos_todos_subamostrado_2.csv")  # Substitua pelo caminho correto

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3051 entries, 0 to 3051
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Data                            3051 non-null   object
 1   Score                           3051 non-null   int64 
 2   Review                          3051 non-null   object
 3   Classificacao_sabia_filtrada    3051 non-null   object
 4   Banco                           3051 non-null   object
 5   Classificacao_sabia_agrupada    3051 non-null   object
 6   Review_Sem_Emojis               3051 non-null   object
 7   Classificacao_Sabia_Agrupada_2  3051 non-null   object
 8   Classificacao_Sabia_Agrupada_3  3051 non-null   object
dtypes: int64(1), object(8)
memory usage: 238.4+ KB


In [3]:
# Elimina o NaN antes de tokenizar
df = df.dropna(subset=["Review_Sem_Emojis"])

In [4]:
# Mapear classes para números
labels = df["Classificacao_Sabia_Agrupada_3"].unique().tolist()
label_map = {label: i for i, label in enumerate(labels)}
df["label"] = df["Classificacao_Sabia_Agrupada_3"].map(label_map)

In [7]:
df[['Classificacao_Sabia_Agrupada_3', 'label']]

,Classificacao_Sabia_Agrupada_3,label
0,Confiança/Orgulho,0
1,Confiança/Orgulho,0
2,Confiança/Orgulho,0
3,Confiança/Orgulho,0
4,Confiança/Orgulho,0
...,...,...
3047,Satisfação,7
3048,Satisfação,7
3049,Satisfação,7
3050,Satisfação,7


In [ ]:
# Separar treino e teste
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["Review_Sem_Emojis"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)

In [ ]:
# Carregar o tokenizer do BERTimbau
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
# Tokenizar os dados
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=MAX_SEQ_LENGTH)

train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

In [ ]:
# Criar Dataset no formato Hugging Face
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "labels": train_labels})
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "labels": val_labels})

In [ ]:
# Definir o modelo BERTimbau para classificação
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Definir métrica de avaliação
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    """Calcula Accuracy e F1-score Macro para avaliação do modelo."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="macro")  # Macro para lidar com classes desbalanceadas

    return {"accuracy": accuracy, "f1_score": f1}

In [ ]:
# Configurar os argumentos de treinamento
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=50,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,  # Mixed Precision para acelerar o treino na GPU do Colab
    push_to_hub=False,
)

In [ ]:
# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-37-ed61f19970b8>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Iniciar o treinamento
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,1.391700,0.814829,0.725041,0.330266
2,0.684400,0.666364,0.770867,0.421018
3,0.569700,0.647054,0.774141,0.466828
4,0.408800,0.600078,0.798691,0.579846
5,0.355900,0.604575,0.801964,0.583052


TrainOutput(global_step=385, training_loss=0.6455845349794859, metrics={'train_runtime': 39.7662, 'train_samples_per_second': 306.793, 'train_steps_per_second': 9.682, 'total_flos': 401265975091200.0, 'train_loss': 0.6455845349794859, 'epoch': 5.0})

In [ ]:
# Salvar o modelo treinado
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("Treinamento concluído! Modelo salvo em:", OUTPUT_DIR)

✅ Treinamento concluído! Modelo salvo em: /content/drive/MyDrive/BERTimbau_Sentimento_Sexto
